In [1]:
#-----------------------------------------------------------------------------------------------------
# 문서들을 embed하는 예제
# - 여기서는 문서들을 전처리하고, 임베딩 하는 과정임.
#
# 질의 응답 시스템 과정
# 문서들 전처리 : 
#    단락별루 분할(\n\n) - 불용어 제거 -문장별루 분할.
# 임베딩 : 
#    kpf-sbert-v1.1로  문장 평균 임베딩벡터 구함 - es에 문장별루 단락text와 평균벡터 저장.
#-----------------------------------------------------------------------------------------------------

import os
import random
import numpy as np
import pandas as pd
import time
import random

from utils import MyUtils
from utils import get_sentences

myutils = MyUtils(yam_file_path='./data/settings.yaml')
myutils.seed_everything()  # seed 설정
DEVICE = myutils.GPU_info() # GPU 혹은 CPU

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30


In [2]:
# DATA_FOLDER에 파일들을 불러와서 DF로 만듬.
# -문서는 \n으로 구분해야 하며, 맨앞에는 title이 와야 하고, \n 다음 문단들이 와야함. 문단들은 \n 구분됨.
# -예: 회사 개요\n{회사내용}\n{제품구성}

# 파일이 여러개인 경우 폴더 지정
DATA_FOLDER = '../../../data11/mpower_doc/사규개정-out-renew/'

files = myutils.getListOfFiles(DATA_FOLDER)
assert len(files) > 0 # files가 0이면 assert 발생
print('*file_count: {}, file_list:{}'.format(len(files), files[0:5]))

# 파일 경로를 지정하면됨.
#files = ["../data11/mpower_doc/신입사원교육.txt",]
docid = 0  # **카운터가 문서에 uid가 되므로, 유일무이한 값므로 지정할것.
count = 0

titles = []
contextids = []
contexts = []

for idx, file_path in enumerate(files):
    if '.ipynb_checkpoints' not in file_path:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = f.read()
            #print(data)
            data_list = data.split('\n\n')  # '\n\n' 구분으로 다락 구분
            docid += 1
            count = 0
            # titles, contextids 임의로 구해서 데이터 프레임 만듬.
            for i in range(len(data_list)):
                if i > 0 and data_list[i]:
                    count += 1
                    titles.append(data_list[0].strip())  # 문서 제목은 맨처음 문장이고 \n\n로 구분되어야 함.
                    contextids.append(f'{docid}_{count}')
                    contexts.append(data_list[0].strip() + '\n' + data_list[i].strip()) # 두번째는 문서 제목+문서내용 합처서 문장만듬.
 
# 데이터 프레임으로 만듬.
df_contexts = pd.DataFrame((zip(contexts, titles, contextids)), columns = ['context','question', 'contextid'])     

print(f'*문단 총 계수:{len(df_contexts)}\n')

*file_count: 92, file_list:['../../../data11/mpower_doc/사규개정-out-renew/위임전결규정.txt', '../../../data11/mpower_doc/사규개정-out-renew/유형자산 관리 지침.txt', '../../../data11/mpower_doc/사규개정-out-renew/의료비 지원 규정.txt', '../../../data11/mpower_doc/사규개정-out-renew/이사회 운영규정.txt', '../../../data11/mpower_doc/사규개정-out-renew/인사 규정.txt']
*문단 총 계수:781



In [3]:
# JSON 파일로 저장해둠.
import json
examples = []

for context, title, contextid in zip(contexts, titles, contextids):
    doc = {}
    doc['context'] = context  # 문단
    doc['title'] = title      # 제목
    doc['contextid'] = contextid # 문서 id
                        
    examples.append(doc)
                        
docs ={}
docs['text'] = examples

#json 파일로 저장.
# JSON 파일을 엽니다.
with open("./data/moco_context.json", "w") as f:
    # 리스트를 JSON으로 변환합니다.
    json.dump(docs, f, indent=4)

In [4]:
# 문장들로 분리
doc_sentences = get_sentences(df=df_contexts, remove_sentnece_len=1, remove_duplication=False)

print(f'len:{len(doc_sentences)}, 1: {doc_sentences[1]}')

  0%|          | 0/781 [00:00<?, ?it/s][Kss]: Oh! You have konlpy.tag.Mecab in your environment. Kss will take this as a backend! :D

100%|██████████| 781/781 [00:06<00:00, 129.80it/s]

*[get_sentences] 문장처리=>len:7, time:6.0210
*[get_sentences] 문장 길이=>평균:6.63764404609475 / MAX: 19 / MIN: 1

len:781, 1: ['위임전결규정', '전결권의 행사', '이 규정에 의하여 전결권자가 결정한 사항은 대표이사가 결정한 것과 동일한 효력을 갖는다.', '전결권자는 이 규정에 의해 위임된 권한을 올바르고 신속하게 행사하여야 하며, 그 결정에 관하여 책임을 진다.', '전결권자는 전결한 사항중 특히 중요하거나 이례적인 사항은 차상위자에게 보고하여야 한다.', '결재단계에 있는 책임자는 신속하게 결정이 내려지도록 최대한 노력을 해야 하며, 타 책임자의 결정을 존중하고 소속 업무에 관한 하위직의 건의와 입안을 고려하여야 한다.', '한건의 문서가 등급이 다른 두개 이상의 전결권에 해당될 때에는 상위 전결권자의 결재를 받는다.', '전결 등급을 낮추기 위하여 한건으로 처리해야 될 문서를 두건 이상으로 분할 처리할 수 없다.']


In [5]:
# 인덱스 추가
from tqdm.notebook import tqdm
from utils import embed_text, bi_encoder, mpower_index_batch, create_index, clustering_embedding, kmedoids_clustering_embedding

# ES 관련
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import bulk
    
# 조건에 맞게 임베딩 처리하는 함수 
def embedding(paragraphs:list)->list:
    # 한 문단에 대한 40개 문장 배열들을 한꺼번에 임베딩 처리함
    embeddings = embed_text(model=BI_ENCODER1, paragraphs=paragraphs, return_tensor=False).astype(FLOAT_TYPE)    
    return embeddings

#---------------------------------------------------------------------------
#문단에 문장들의 임베딩을 구하여 각각 클러스터링 처리함.
#---------------------------------------------------------------------------
def index_data(es, df_contexts, doc_sentences:list):
    #클러스터링 계수는 문단의 계수보다는 커야 함. 
    #assert num_clusters <= len(doc_sentences), f"num_clusters:{num_clusters} > len(doc_sentences):{len(doc_sentences)}"
    #-------------------------------------------------------------
    # 각 문단의 문장들에 벡터를 구하고 리스트에 저장해 둠.
    start = time.time()
    cluster_list = []

    rfile_names = df_contexts['contextid'].values.tolist()
    rfile_texts = df_contexts['context'].values.tolist()

    if OUT_DIMENSION == 0:
        dimension = 768
    else:
        dimension = 128

    clustering_num = NUM_CLUSTERS
        
    docs = []
    count = 0
    for i, sentences in enumerate(tqdm(doc_sentences)):
        embeddings = embedding(sentences)
        if i < 3:
            print(f'[{i}] sentences-------------------')
            if len(sentences) > 5:
                print(sentences[:5])
            else:
                print(sentences)
                
            myutils.log_message(f'*[index_data] embeddings.shape: {embeddings.shape}', log_folder='./log/documet')
            print()
        
        #----------------------------------------------------------------
        multiple = 1
        
        # [bong][2023-04-28] 임베딩 출력 계수에 따라 클러스터링 계수를 달리함.
        if NUM_CLUSTERS_VARIABLE == True:
            embeddings_len = embeddings.shape[0]
            if embeddings_len > 2000:
                multiple = 6
            elif embeddings_len > 1000:
                multiple = 5 # 5배
            elif embeddings_len > 600:
                multiple = 4 # 4배
            elif embeddings_len > 300:
                multiple = 3 # 3배
            elif embeddings_len > 100:
                multiple = 2 # 2배
        #----------------------------------------------------------------
        
        # 0=문장클러스터링 임베딩
        if EMBEDDING_METHOD == 0:
            if CLUSTRING_MODE == "kmeans":
                # 각 문단에 분할한 문장들의 임베딩 값을 입력해서 클러스터링 하고 평균값을 구함.
                # [bong][2023-04-28] 문장이 많은 경우에는 클러스터링 계수를 2,3배수로 함
                emb = clustering_embedding(embeddings = embeddings, outmode=OUTMODE, num_clusters=(clustering_num*multiple), seed=SEED).astype(FLOAT_TYPE) 
            else:
                emb = kmedoids_clustering_embedding(embeddings = embeddings, outmode=OUTMODE, num_clusters=(clustering_num*multiple), seed=SEED).astype(FLOAT_TYPE) 
            
        # 1= 문장평균임베딩
        elif EMBEDDING_METHOD == 1:
            # 문장들에 대해 임베딩 값을 구하고 평균 구함.
            arr = np.array(embeddings).astype(FLOAT_TYPE)
            emb = arr.mean(axis=0).reshape(1,-1) #(128,) 배열을 (1,128) 형태로 만들기 위해 reshape 해줌
            clustering_num = 1  # 평균값일때는 NUM_CLUSTERS=1로 해줌.
        # 2=문장임베딩
        else:
            emb = embeddings

        if i < 3:
            myutils.log_message(f'*[index_data] cluster emb.shape: {emb.shape}', log_folder='./log/documet')
            print()
        
        #--------------------------------------------------- 
        # docs에 저장 
        #  [bong][2023-04-28] 여러개 벡터인 경우에는 벡터를 10개씩 분리해서 여러개 docs를 만듬.
        for j in range(multiple):
            count += 1
            doc = {}                                #dict 선언
            doc['rfile_name'] = rfile_names[i]      # contextid 담음
            doc['rfile_text'] = rfile_texts[i]      # text 담음.
            doc['dense_vectors'] = emb[j * clustering_num : (j+1) * clustering_num] # emb 담음.
            docs.append(doc)
        #---------------------------------------------------    

            if count % BATCH_SIZE == 0:
                mpower_index_batch(es, ES_INDEX_NAME, docs, vector_len=clustering_num, dim_size=dimension)
                docs = []
                myutils.log_message("[index_data](1) Indexed {} documents.".format(count), log_folder='./log/documet')
                 

    if docs:
        mpower_index_batch(es, ES_INDEX_NAME, docs, vector_len=clustering_num, dim_size=dimension)
        myutils.log_message("[index_data](2) Indexed {} documents.".format(count), log_folder='./log/documet')

    es.indices.refresh(index=ES_INDEX_NAME)

    myutils.log_message(f'*인덱싱 시간 : {time.time()-start:.4f}\n', log_folder='./log/documet')
    print()
#---------------------------------------------------------------------------


In [6]:
# param--------------------------------------------------------------------
OUT_DIMENSION =128   # 128 혹은 768이면 0입력
EMBEDDING_METHOD=0  # 0=클러스터링 임베딩, 1=문장평균임베딩, 2=문장임베딩
NUM_CLUSTERS=10     # 클러스터링 임베딩일때 클러스터링 수 
NUM_CLUSTERS_VARIABLE=False # 클러스터링 임베딩일때 클러스터링을 문장계수마다 다르계할지.
CLUSTRING_MODE = "kmeans"  # "kmeans" = k-평균 군집 분석, kmedoids =  k-대표값 군집 분석
OUTMODE = "mean"           # kmeans 일때=>mean=평균임베딩값, max=최대임베딩값.,  kmedoids 일때 =>mean=평균임베딩값, medoid=대표임베딩값.

MODEL_PATH = '../../../data11/model/kpf-sbert-128d-v1'#'../../../data11/model/kpf-sbert-v1.1'
POLLING_MODE = 'mean' # 폴링모드 

FLOAT_TYPE = 'float16' # float32 혹은 float16
SEED = 111

# ES 접속
ES_URL = 'http://10.10.4.10:9200/'               # es 접속 주소
ES_INDEX_NAME = 'qaindex_128_10'          # 생성혹은 추가할 인덱스 명
ES_INDEX_FILE = './data/mpower10u_128d_10.json'   # 인덱스 구조 파일경로
BATCH_SIZE=20       # ES 배치 사이즈
CREATE_INDEX = True # True이면 기존에 인덱스가 있다면 제거하고 다시 생성.
# param--------------------------------------------------------------------

es = Elasticsearch(ES_URL)
create_index(es, ES_INDEX_FILE, ES_INDEX_NAME, create=CREATE_INDEX)

# 임베딩 모델 로딩
WORD_EMBDDING_MODEL1, BI_ENCODER1 = bi_encoder(model_path=MODEL_PATH, max_seq_len=512, do_lower_case=True, 
                                               pooling_mode=POLLING_MODE, out_dimension=OUT_DIMENSION, device=DEVICE)

print(BI_ENCODER1)
print()
try:
    index_data(es, df_contexts, doc_sentences)
except Exception as e:
    error = f'index_data fail'
    msg = f'{error}=>{e}'
    myutils.log_message(f'/embed/es {msg}', log_folder='./log/documet')

/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


1:{			"settings": {
				"number_of_shards": 2,
				"number_of_replicas": 1,
				"index": {
					"merge": {
						"scheduler": {
							"max_thread_count": 1
						}
					}
				},
				"analysis": {
					"analyzer": {
						"mpower10u_korean_analyzer": {
							"type": "custom",
							"tokenizer": "mpower10u_korean_tokenizer",
							"filter": [
								"lowercase",
								"stop",
								"nori_readingform",
								"mpower10u_korean_pos"
							]
						}
					},
					"tokenizer": {
						"mpower10u_korean_tokenizer": {
							"type": "nori_tokenizer",
							"decompound_mode": "discard"
						}
					},
					"filter": {
						"mpower10u_korean_pos": {
							"type": "nori_part_of_speech",
							"stoptags": [
								"J", "E", "XSA", "XSN", "XSV"
							]
						}
					}
				}
			},
			"mappings": {
				"_source": {
					"enabled": true
				},
				"dynamic":"strict",
				"properties": {
					"rfile_name": {
						"type": "text",
						"store": true
					},
					"rfile_text": {
						"ty

  0%|          | 0/781 [00:00<?, ?it/s]

[0] sentences-------------------
['위임전결규정', '적용범위', '주식회사 모코엠시스(이하 회사라 한다)의 업무수행에 관련된 모든 권한과 책임은 다른 사규로 정하지 않는 한 이 규정을 따르고 이 규정에 정하지 않는 권한과 책임은 대표이사가 정한다.', '목적', '이 규정은 회사 업무 집행에 관련된 권한의 한계와 책임의 소재를 분명히 하여 업무수행의 신속과 효율 제고를 도모함을 목적으로 한다.']
[2023-12-01 16:29:14]*[index_data] embeddings.shape: (7, 128)


[2023-12-01 16:29:14]*[index_data] cluster emb.shape: (7, 128)


[1] sentences-------------------
['위임전결규정', '전결권의 행사', '이 규정에 의하여 전결권자가 결정한 사항은 대표이사가 결정한 것과 동일한 효력을 갖는다.', '전결권자는 이 규정에 의해 위임된 권한을 올바르고 신속하게 행사하여야 하며, 그 결정에 관하여 책임을 진다.', '전결권자는 전결한 사항중 특히 중요하거나 이례적인 사항은 차상위자에게 보고하여야 한다.']
[2023-12-01 16:29:14]*[index_data] embeddings.shape: (8, 128)


[2023-12-01 16:29:14]*[index_data] cluster emb.shape: (8, 128)


[2] sentences-------------------
['위임전결규정', '전결권자의 권한', '대표이사는 회사의 최고경영자로서 이사회에서 정한 업무의 집행과 전사적인 기본 경영방침 및 중요한 업무를 결정, 집행한다.', '본부장은 사업본부의 사업계획을 수립, 시행하여 업무를 총괄집행하고 책임지며 회사의 기본방침 수립과 업무 추진을 위하여 최고 경영층을 보좌한다.', '부서(팀)장은 소관부서(팀)의 사업계획과 예산을 입안하고 집행하며, 소관부서(팀)를대표하여 업무를 처리하고 소속직원을 지

  0%|          | 0/20 [00:00<?, ?it/s]

...

[2023-12-01 16:29:43][index_data](1) Indexed 760 documents.



  0%|          | 0/20 [00:00<?, ?it/s]

[2023-12-01 16:29:44][index_data](1) Indexed 780 documents.



  0%|          | 0/1 [00:00<?, ?it/s]

[2023-12-01 16:29:44][index_data](2) Indexed 781 documents.

[2023-12-01 16:29:44]*인덱싱 시간 : 32.7799



